<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/transformers_demo/blip_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install transformers
%pip install torch
%pip install ipython
%pip install ipywidgets

In [ ]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

blip_processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

In [ ]:
from urllib.request import urlopen
from PIL import Image

puppy_path = "https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/chapter09/images/puppy.png"
image = Image.open(urlopen(puppy_path)).convert("RGB")

In [ ]:
from IPython.display import HTML, display
import ipywidgets as widgets

def text_eventhandler(*args):
  question = args[0]["new"]
  if question:
    args[0]["owner"].value = ""

    if not memory:
      prompt = " Question: " + question + " Answer:"
    else:
      template = "Question: {} Answer: {}."
      prompt = " ".join([
          template.format(memory[i][0],memory[i][1]) for i in range(len(memory))
      ]) + " Question: " + question + " Answer:"

    inputs = blip_processor(image, text=prompt, return_tensors="pt").to(device, torch.float16)

    generated_ids = model.generate(**inputs, max_new_tokens=100)
    generated_text = blip_processor.batch_decode(generated_ids, skip_spceial_tokens=True)
    print(generated_text[0].strip())
    generated_text = generated_text[0].strip().split("Question")[0]
    print(generated_text)

    memory.append((question, generated_text))

    output.append_display_data(HTML("<b>USER: </b>" + question))
    output.append_display_data(HTML("<b>BLIP-2: </b> "+ generated_text))
    output.append_display_data(HTML("<br/>"))

memory = []
in_text = widgets.Text()
in_text.continuous_update = False
in_text.observe(text_eventhandler, "value")
output = widgets.Output()

In [ ]:
display(
  widgets.VBox(
      children=[output, in_text],
      layout=widgets.Layout(display="inline-flex", flex_flow="column-reverse"),
  )
)